In [1]:
# plaidml

import plaidml.keras
plaidml.keras.install_backend()

In [2]:
# Data Loader

import csv
import cv2
import numpy as np
from PIL import Image
from keras.preprocessing import image
from keras.utils import Sequence, to_categorical

def load_csv(folder, csvname):
    # Read CSV Data
    _file = open('{}/{}'.format(folder, csvname), 'r', encoding='utf-8')
    _reader = csv.reader(_file)
    
    filenames = []
    labels = {}
    for line in _reader:
        _x = '{}/{}'.format(folder, line[0])
        _y = [int(it) for it in line[1:]]
        filenames.append(_x)
        labels[_x] = _y
    _file.close()
    
    return filenames, labels

class SVHNLoader(Sequence):
    def __init__(self, filenames, labels, batch_size=32, dim=(48, 48), n_channels=3,
                 shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.filenames = filenames
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.filenames) / self.batch_size))
    
    def __getitem__(self, index):
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        filenames_temp = [self.filenames[k] for k in indexes]

        # Generate data
        X, Y = self.__data_generation(filenames_temp)

        return X, Y
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.filenames))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __data_generation(self, filenames_temp):
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        Y = [[], [], [], [], [], []]

        # Generate data
        for idx, filename in enumerate(filenames_temp):
            # Store sample
            X[idx,] = self.__preprocess_image(filename)

            # Store class
            _Y = self.labels[filename]
            for it in range(6):
                Y[it].append(_Y[it])

        return X, Y
    
    def __preprocess_image(self, filename):
        '''
        Image Preprocessing (grayscale)
        '''
        img = image.load_img(filename, target_size=(48, 48))
        img_tensor = image.img_to_array(img)
        img_tensor = np.dot(img_tensor[...,:3], [0.299, 0.587, 0.114])
        img_tensor = np.squeeze(img_tensor)
        img_tensor /= 255.0
        img_tensor = img_tensor - img_tensor.mean()
        img_tensor = np.reshape(img_tensor, (48, 48, 1))
        img_tensor = np.repeat(img_tensor, 3, axis=2)
        return img_tensor

In [3]:
# Keras

import keras
import tensorflow as tf
from keras.callbacks import EarlyStopping
from keras.layers import Input, BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Dense, Dropout, Flatten
from keras import optimizers

In [4]:
from keras.applications import VGG16

def vgg_based_model():
    pretrained_vgg = VGG16(include_top=False, input_shape=(48, 48, 3))
    M = pretrained_vgg.output
    
    # FC Layer
    Mout = Flatten()(M)
    Mout = Dense(2048, activation='relu', name='FC1')(Mout)
    Mout = Dense(1024, activation='relu', name='FC2')(Mout)
    Mout = Dense(1024, activation='relu', name='FC3')(Mout)
    
    # Softmax
    out_hasNum = Dense(2, activation='softmax', name='SM_hasNum')(Mout)
    out_digitLen = Dense(5, activation='softmax', name='SM_digitLen')(Mout)
    out_digit1 = Dense(11, activation='softmax', name='SM_digit1')(Mout)
    out_digit2 = Dense(11, activation='softmax', name='SM_digit2')(Mout)
    out_digit3 = Dense(11, activation='softmax', name='SM_digit3')(Mout)
    out_digit4 = Dense(11, activation='softmax', name='SM_digit4')(Mout)
    out = [out_hasNum, out_digitLen, out_digit1, out_digit2, out_digit3, out_digit4]
    
    # Model
    optim = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001, amsgrad=True)
    model = keras.Model(inputs=pretrained_vgg.input, outputs=out)
    loss = {
        'SM_hasNum': 'sparse_categorical_crossentropy',
        'SM_digitLen': 'sparse_categorical_crossentropy',
        'SM_digit1': 'sparse_categorical_crossentropy',
        'SM_digit2': 'sparse_categorical_crossentropy',
        'SM_digit3': 'sparse_categorical_crossentropy',
        'SM_digit4': 'sparse_categorical_crossentropy'
    }
    model.compile(loss=loss, optimizer=optim, metrics=['accuracy'])
    
    return model


In [20]:
filenames = {'train': None, 'test': None}
labels = {'train': None, 'test': None}
filenames['train'], labels['train'] = load_csv('./datasets/train_new', 'labels.csv')
filenames['test'], labels['test'] = load_csv('./datasets/test_new', 'labels.csv')

train_dl = SVHNLoader(filenames['train'], labels['train'], batch_size=64)
test_dl = SVHNLoader(filenames['test'], labels['test'], batch_size=64)

In [5]:
model = vgg_based_model()
model.summary()

from keras.utils.vis_utils import plot_model
plot_model(model, to_file='./model_plot.png', show_shapes=True, show_layer_names=True)

callbacks = [
    EarlyStopping(monitor='val_loss')
]

# model1.fit_generator(generator=train_dl,
#                     validation_data=test_dl,
#                     use_multiprocessing=True,
#                     workers=6,
#                     epochs=25,
#                     callbacks=callbacks,
#                     verbose=0)


# model.fit_generator(generator=train_dl,
#                     validation_data=test_dl,
#                     epochs=25,
#                     callbacks=callbacks)

INFO:plaidml:Opening device "opencl_amd_ellesmere.0"


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 48, 48, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 48, 48, 64)   1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 48, 48, 64)   36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 24, 24, 64)   0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [7]:
# Save Model
model.save('./svhn_model.h5')

In [48]:
import time

def read_img(filename):
    img = image.load_img(filename, target_size=(48, 48))
    img_tensor = image.img_to_array(img)
    img_tensor = np.squeeze(img_tensor)
    img_tensor /= 255.
    img_tensor = img_tensor - img_tensor.mean()
    return img_tensor

def predict(filename):
    img = read_img(filename)
    dat = model.predict(np.array([img]))
    # return dat
    o1 = np.argmax(dat[0])
    o2 = np.argmax(dat[1])
    o3 = np.argmax(dat[2])
    o4 = np.argmax(dat[3])
    o5 = np.argmax(dat[4])
    o6 = np.argmax(dat[5])
    return [o1, o2, o3, o4, o5, o6]

def predict_with_time(folder, filename):
    t1 = time.time()
    output = predict('{}/{}'.format(folder, filename))
    t2 = time.time()
    
    print('=' * 12, filename, '=' * 12)
    print('Raw Output :', output)
    print('Elapsed Time :', round(t2 - t1, 6))
    

folder = './datasets/khudoors/test'
predict_with_time(folder, '12022.png')

============ 12022.png ============
Raw Output : [1, 4, 3, 8, 10, 2]
Elapsed Time : 0.015003
